## Data prep.

In [ ]:
# import packages
import pandas as pd
import numpy as np
from numpy import where
from sklearn.impute import KNNImputer
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from warnings import simplefilter
from imblearn.under_sampling import RandomUnderSampler
simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.pipeline import Pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# connect the drive
df = pd.read_excel('/content/drive/MyDrive/MMAI/Finance/Bankruptcy_data_Final.xlsx')
#df = pd.read_csv('Bankruptcy_data_final.csv')

In [ ]:
# look at shape
print(df.shape)
df.head(3)

(92872, 13)


,EPS,Liquidity,Profitability,Productivity,Leverage Ratio,Asset Turnover,Operational Margin,Return on Equity,Market Book Ratio,Assets Growth,Sales Growth,Employee Growth,BK
0,1.58,0.36,0.18,0.13,1.33,1.77,0.07,0.15,2.22,NaN,NaN,NaN,0
1,1.41,0.36,0.19,0.12,1.31,1.59,0.07,0.13,2.41,0.126319,0.014278,0.040179,0
2,0.31,0.32,0.13,0.08,1.03,1.55,0.05,0.04,2.56,0.368077,0.327909,0.566524,0


In [ ]:
print('Full dataset target distribution...')
print('1:', df['BK'][df['BK']==1].count(),',',df['BK'][df['BK']==1].count()/df['BK'].count() )
print('0:', df['BK'][df['BK']==0].count(),',',df['BK'][df['BK']==0].count()/df['BK'].count() )

Full dataset target distribution...
1: 558 , 0.00600826944611939
0: 92314 , 0.9939917305538806


In [ ]:
# check out the columns
print('Understanding columns...')
for col in df.columns:
  print(col,':')
  print('Total nulls:',df[col].isnull().values.sum())
  print('Min:',df[col].min())
  print('Max:',df[col].max())
  if df[col].isnull().values.sum() == 0:
    print('All good for', col)
  elif df[col].isnull().values.sum() <= 300:
    print('Drop the nulls for', col,'\n')
  else:
    print('Use imputation for', col,'\n')

Understanding columns...
EPS :
Total nulls: 5
Min: -384000.0
Max: 55339.0
Drop the nulls for EPS 

Liquidity :
Total nulls: 247
Min: -25968.52
Max: 1.0
Drop the nulls for Liquidity 

Profitability :
Total nulls: 247
Min: -79682.0
Max: 140.58
Drop the nulls for Profitability 

Productivity :
Total nulls: 247
Min: -5093.0
Max: 1102.0
Drop the nulls for Productivity 

Leverage Ratio :
Total nulls: 26
Min: -7811.0
Max: 75970.38
Drop the nulls for Leverage Ratio 

Asset Turnover :
Total nulls: 247
Min: -31.59
Max: 276.38
Drop the nulls for Asset Turnover 

Operational Margin :
Total nulls: 5557
Min: -30175.7
Max: 394.47
Use imputation for Operational Margin 

Return on Equity :
Total nulls: 8
Min: -88875.14
Max: 39500.0
Drop the nulls for Return on Equity 

Market Book Ratio :
Total nulls: 57
Min: -3151500.0
Max: 3455419.33
Drop the nulls for Market Book Ratio 

Assets Growth :
Total nulls: 6701
Min: -1.0
Max: 14231.0
Use imputation for Assets Growth 

Sales Growth :
Total nulls: 6701
Min: 

In [ ]:
# drop nulls where less than 300 
print('All:',df.shape)
df = df.dropna(subset=['Return on Equity','Leverage Ratio','EPS','Liquidity',
                       'Profitability','Leverage Ratio','Asset Turnover'])
print('Nulls dropped where very few:',df.shape)

# Remove outliers 
df_a = df[(df['Employee Growth']<=1000) | (df['Employee Growth'].isnull())] 
df_a = df_a[(df_a['Sales Growth']<=1000) | (df_a['Sales Growth'].isnull())] 
df_a = df_a[(df_a['Sales Growth']>=-10) | (df_a['Sales Growth'].isnull())] 
df_a = df_a[(df_a['Assets Growth']<=4000) | (df_a['Assets Growth'].isnull())] 
df_a = df_a[(df_a['Return on Equity']<=10000) | (df_a['Return on Equity'].isnull())] 
df_a = df_a[(df_a['Return on Equity']>=-25000) | (df_a['Return on Equity'].isnull())] 
df_a = df_a[(df_a['Operational Margin']<=11) | (df_a['Operational Margin'].isnull())] 
df_a = df_a[(df_a['Asset Turnover']>=-2) | (df_a['Asset Turnover'].isnull())]  
df_a = df_a[(df_a['Leverage Ratio']<=3500) | (df_a['Leverage Ratio'].isnull())]
df_a = df_a[(df_a['Leverage Ratio']>=-3500) | (df_a['Leverage Ratio'].isnull())]
df_a = df_a[(df_a['Productivity']<=50) |  (df_a['Productivity'].isnull())]
df_a = df_a[(df_a['Profitability']<=6) |  (df_a['Profitability'].isnull())]  
df_a = df_a[(df_a['Liquidity']>=-15000)  |  (df_a['Liquidity'].isnull())]  
print('Outliers dropped, none were target or null = 1:',df_a.shape)
print('Remaining target:',df_a['BK'].sum())

All: (92607, 13)
Nulls dropped where very few: (92607, 13)
Outliers dropped, none were target or null = 1: (92568, 13)
Remaining target: 554


# Split Train/Test

In [ ]:
# split training data
y = df_a['BK']
X = df_a.drop(columns=['BK'])

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.15,
                                                    random_state=42)
print('Training:',X_train.shape)
print('Testing:',X_test.shape)

Training: (78682, 12)
Testing: (13886, 12)


# Scale/Fill Missing Values/Sampling

# Scale

In [ ]:
#mm_scaler = MinMaxScaler()
scaler = PowerTransformer(method='yeo-johnson')
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train),columns=X_train.columns)

# Fill Missing Values

In [ ]:
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter=10,random_state=42)
imp.fit(X_train_scaled)
X_train_imp = pd.DataFrame(imp.transform(X_train_scaled),columns=X_train.columns)

In [ ]:
# look at the imputed version
X_train_imp.head(3)

,EPS,Liquidity,Profitability,Productivity,Leverage Ratio,Asset Turnover,Operational Margin,Return on Equity,Market Book Ratio,Assets Growth,Sales Growth,Employee Growth
0,-0.001569,-0.510463,0.049357,0.022965,0.007204,0.439325,0.038629,-0.009368,-0.017929,-0.248105,-0.391908,-0.664258
1,-0.000195,0.148403,0.088999,0.157606,-0.004092,0.005036,0.278746,0.006716,0.011993,-0.042717,0.063784,0.103333
2,-0.001431,0.747505,-1.786394,-0.519446,0.019240,-1.332720,-2.049985,0.002444,0.024358,-2.033878,-0.153317,-0.174993


In [ ]:
# see shapes
counter = Counter(y_train)
print('Training distribution:',counter)
print('1:', y_train[y_train==1].count()/y_train.count() )
print('0:', y_train[y_train==0].count()/y_train.count() )

ycounter = Counter(y_test)
print('Testing distribution:',ycounter)
print('1:', y_test[y_test==1].count()/y_test.count() )
print('0:', y_test[y_test==0].count()/y_test.count() )

Training distribution: Counter({0: 78197, 1: 485})
1: 0.006164052769375461
0: 0.9938359472306245
Testing distribution: Counter({0: 13817, 1: 69})
1: 0.004969033558980268
0: 0.9950309664410197


In [ ]:
# New features
def pos(x):
  if x > 0:
    y = 1
  else: 
    y = 0
  return y

def beyond_one(x):
  if x >= 1:
    y = 1
  else: 
    y = 0
  return y

#X_train_imp['Pos_Liquidity'] = X_train_imp['Liquidity'].apply(pos)
#X_train_imp['Pos_Profitability'] = X_train_imp['Profitability'].apply(pos)
#X_train_imp['High_Leverage'] = X_train_imp['Leverage Ratio'].apply(beyond_one)


# Oversampling

In [ ]:
# transform the training dataset with smote /and under sampling
oversample = SMOTE(sampling_strategy=0.05) #.1
X_smote, y_smote = oversample.fit_resample(X_train_imp, y_train)
counter = Counter(y_smote)
print('New distribution:',counter)
undersample = RandomUnderSampler(sampling_strategy=0.07) #.7
X_smote_under, y_smote_under = undersample.fit_resample(X_smote, y_smote)
counter = Counter(y_smote_under)
print('New distribution:',counter)

New distribution: Counter({0: 78197, 1: 3909})
New distribution: Counter({0: 55842, 1: 3909})


## Modeling

In [ ]:
# base model
Pipe= Pipeline([('rf', RandomForestClassifier())])
param_grid = {'rf__criterion': ['entropy']}
model = GridSearchCV(Pipe,param_grid,cv=5,scoring='roc_auc',n_jobs=-1)
model.fit(X_smote_under,y_smote_under) 
print('\n\nBest Score:', model.best_score_,'\n')
print('Best parameters:', model.best_params_)



Best Score: 0.9935219894398202 

Best parameters: {'rf__criterion': 'entropy'}


In [ ]:
from sklearn.metrics import roc_auc_score

#impute with iterative imputer and add features
X_test_scaled = scaler.transform(X_test)
X_test_imp = imp.transform(X_test_scaled)
X_test_imp  = pd.DataFrame(X_test_imp , columns = X_test.columns )
#add features
#X_test_imp['Pos_Liquidity'] = X_test_imp['Liquidity'].apply(pos)
#X_test_imp['Pos_Profitability'] = X_test_imp['Profitability'].apply(pos)
#X_test_imp['High_Leverage'] = X_test_imp['Leverage Ratio'].apply(beyond_one)

#predict
pred = model.predict_proba(X_test_imp)[:,1]
threshold = 0.1
print('Confusion matrix:\n',confusion_matrix(y_test, pred>threshold))
print('\nClassification report:\n',classification_report(y_test, pred>threshold))
print (f'AUC Score: {roc_auc_score(y_test,pred>threshold)}')

Confusion matrix:
 [[12847   970]
 [   17    52]]

Classification report:
               precision    recall  f1-score   support

           0       1.00      0.93      0.96     13817
           1       0.05      0.75      0.10        69

    accuracy                           0.93     13886
   macro avg       0.52      0.84      0.53     13886
weighted avg       0.99      0.93      0.96     13886

AUC Score: 0.8417099078744626


Tuned RF

In [ ]:
Pipe= Pipeline([('rf', RandomForestClassifier())])
param_grid = {'rf__criterion': ['entropy'],#,'gini'],
              #'rf__n_estimators': (40,100),
              'rf__min_samples_leaf': (3,6,None),
              'rf__max_depth': (5,8,None),
              'rf__class_weight': ['balanced','balanced_subsample'],
              'rf__max_features':['auto', 'sqrt', 'log2']
              }
model = GridSearchCV(Pipe,param_grid,cv=5,scoring='roc_auc',n_jobs=-1)
model.fit(X_smote_under,y_smote_under) 
print('\n\nBest Score:', model.best_score_,'\n')
print('Best parameters:', model.best_params_)



Best Score: 0.9924543718559418 

Best parameters: {'rf__class_weight': 'balanced', 'rf__criterion': 'entropy', 'rf__max_depth': None, 'rf__max_features': 'sqrt', 'rf__min_samples_leaf': 3}


In [ ]:
from sklearn.metrics import roc_auc_score
X_test_scaled = scaler.transform(X_test)
X_test_imp = imp.transform(X_test_scaled)
X_test_imp  = pd.DataFrame(X_test_imp , columns = X_test.columns )
pred = model.predict_proba(X_test_imp)[:,1]
threshold = 0.1
print('Confusion matrix:\n',confusion_matrix(y_test, pred>threshold))
print('\nClassification report:\n',classification_report(y_test, pred>threshold))
print (f'AUC Score: {roc_auc_score(y_test,pred>threshold)}')

Confusion matrix:
 [[12467  1350]
 [   15    54]]

Classification report:
               precision    recall  f1-score   support

           0       1.00      0.90      0.95     13817
           1       0.04      0.78      0.07        69

    accuracy                           0.90     13886
   macro avg       0.52      0.84      0.51     13886
weighted avg       0.99      0.90      0.94     13886

AUC Score: 0.8424514854102224


Tuned Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
# base model
nb = GaussianNB()
Pipe= Pipeline([('nb', nb)])
param_grid = {'nb__var_smoothing': (0.000000001,.00000000001,.0000000001),
              }
model = GridSearchCV(Pipe,param_grid,cv=5,scoring='roc_auc',n_jobs=-1)
model.fit(X_smote_under,y_smote_under) 
print('\n\nBest Score:', model.best_score_,'\n')
print('Best parameters:', model.best_params_)



Best Score: 0.8409268153507246 

Best parameters: {'nb__var_smoothing': 1e-11}


In [ ]:
X_test_scaled = scaler.transform(X_test)
X_test_imp = imp.transform(X_test_scaled)
X_test_imp  = pd.DataFrame(X_test_imp , columns = X_test.columns )
pred = model.predict_proba(X_test_imp)[:,1]
threshold = 0.1
print('Confusion matrix:\n',confusion_matrix(y_test, pred>threshold))
print('\nClassification report:\n',classification_report(y_test, pred>threshold))
print (f'AUC Score: {roc_auc_score(y_test,pred>threshold)}')

Confusion matrix:
 [[ 2909 10908]
 [    5    64]]

Classification report:
               precision    recall  f1-score   support

           0       1.00      0.21      0.35     13817
           1       0.01      0.93      0.01        69

    accuracy                           0.21     13886
   macro avg       0.50      0.57      0.18     13886
weighted avg       0.99      0.21      0.35     13886

AUC Score: 0.5690369876218437


Logistic Regression

In [ ]:
#more to tune -> could be better to use for explanation
from sklearn.linear_model import LogisticRegression
# base model
lr = LogisticRegression()
Pipe= Pipeline([('lr', lr)]) #no tuning
param_grid = {'lr__penalty': ['l2'],
              }
model = GridSearchCV(Pipe,param_grid,cv=5,scoring='roc_auc',n_jobs=-1)
model.fit(X_smote_under,y_smote_under) 
print('\n\nBest Score:', model.best_score_,'\n')
print('Best parameters:', model.best_params_)



Best Score: 0.7930714125367965 

Best parameters: {'lr__penalty': 'l2'}


In [ ]:
X_test_scaled = scaler.transform(X_test)
X_test_imp = imp.transform(X_test_scaled)
X_test_imp  = pd.DataFrame(X_test_imp , columns = X_test.columns )
pred = model.predict_proba(X_test_imp)[:,1]
threshold = 0.1
print('Confusion matrix:\n',confusion_matrix(y_test, pred>threshold))
print('\nClassification report:\n',classification_report(y_test, pred>threshold))
print (f'AUC Score: {roc_auc_score(y_test,pred>threshold)}')

Confusion matrix:
 [[12324  1493]
 [   35    34]]

Classification report:
               precision    recall  f1-score   support

           0       1.00      0.89      0.94     13817
           1       0.02      0.49      0.04        69

    accuracy                           0.89     13886
   macro avg       0.51      0.69      0.49     13886
weighted avg       0.99      0.89      0.94     13886

AUC Score: 0.692349164492806


KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest

KNN = KNeighborsClassifier()
skb = SelectKBest()
Pipe= Pipeline([#('skb',skb),
                   ('KNN',KNN)
                   ])
param_grid = {#'skb__k':(5,7,9,11,13),     
              'KNN__weights': ('uniform','distance'),
              #'KNN__algorithm': ('auto', 'ball_tree', 'kd_tree', 'brute'),
              'KNN__metric': ('euclidean','manhattan','chebyshev','minkowski')
              }
model = GridSearchCV(Pipe,param_grid,cv=5,scoring='roc_auc',n_jobs=-1)
model.fit(X_smote_under,y_smote_under) 
print('\n\nBest Score:', model.best_score_,'\n')
print('Best parameters:', model.best_params_)



Best Score: 0.9892021679248801 

Best parameters: {'KNN__metric': 'manhattan', 'KNN__weights': 'distance'}


In [ ]:
X_test_scaled = scaler.transform(X_test)
X_test_imp = imp.transform(X_test_scaled)
X_test_imp  = pd.DataFrame(X_test_imp , columns = X_test.columns )
pred = model.predict_proba(X_test_imp)[:,1]
threshold = 0.1
print('Confusion matrix:\n',confusion_matrix(y_test, pred>threshold))
print('\nClassification report:\n',classification_report(y_test, pred>threshold))
print (f'AUC Score: {roc_auc_score(y_test,pred>threshold)}')

Confusion matrix:
 [[12917   900]
 [   36    33]]

Classification report:
               precision    recall  f1-score   support

           0       1.00      0.93      0.97     13817
           1       0.04      0.48      0.07        69

    accuracy                           0.93     13886
   macro avg       0.52      0.71      0.52     13886
weighted avg       0.99      0.93      0.96     13886

AUC Score: 0.7065618598386989
